# Lesson 3: Data Validation and Error Handling

# Overview of Data Validation and Error Handling

Welcome back! Now that you've effectively learned how to secure your routes using middleware, let's dive into the next crucial aspect of building a robust To-Do list application — **Data Validation and Error Handling**. While we covered validation in previous lessons, in this unit, we'll enhance our application by adding data validation techniques.

## What You'll Learn

### ✅ Implementing Data Validation

We'll explore how to implement validation in Django models using validators. For example, validating the length of a task:

```python
from django.db import models
from django.core.exceptions import ValidationError
from django.contrib.auth.models import User

def validate_todo_length(value):
    if len(value) < 5:
        raise ValidationError('Task must be at least 5 characters long.')

class Todo(models.Model):
    task = models.CharField(max_length=200, validators=[validate_todo_length])
    completed = models.BooleanField(default=False)
    user = models.ForeignKey(User, on_delete=models.CASCADE)

    def __str__(self):
        return self.task
```

In the code snippet above, we have a `Todo` model with a `task` field that has a maximum length of 200 characters. Additionally, we define a custom validator `validate_todo_length`, which checks if the task is at least 5 characters long. If the validation fails, a `ValidationError` is raised.

### ✅ Handling Errors Gracefully

Next, let's learn how to catch and handle errors to provide meaningful feedback to users. Below is an example of how to handle validation errors in views:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from django.core.exceptions import ValidationError
from .models import Todo
import json

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        task = data.get('task')

        if not task:
            return JsonResponse({'error': 'Task is required'}, status=400)

        new_todo = Todo.objects.create(task=task, user=request.user)

        try:
            new_todo.full_clean()
            new_todo.save()
            return JsonResponse({'message': 'Todo added successfully'}, status=201)
        except ValidationError as e:
            return JsonResponse({'error': e.message_dict}, status=400)
    
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

In the example above, the `add_todo` view handles adding a new task. It checks whether the request is a `POST` method and extracts the task from the request data. If the task is missing, an error response is returned.

We create a `Todo` object and call the `full_clean()` method to validate the instance. This method checks all validators and raises a `ValidationError` if validation fails. If any validation errors occur, they are caught, and error messages are returned to the user.

### 📝 Key Points

- **Validation Check**: Custom validation logic ensures tasks are at least 5 characters long.
- **Handling Errors**: The `full_clean()` method validates the model fields, catching validation errors and returning user-friendly error messages.
- **Success Feedback**: If validation succeeds, a success message is returned along with the HTTP status code `201` to indicate successful creation.

## Why It Matters

### 🚀 Ensures Data Integrity
Valid data ensures smooth application performance and prevents potential issues.

### 🙌 Enhances User Experience
By catching and displaying friendly error messages, users are guided to correct their inputs, making the app more user-friendly.

### 🔐 Improves Security
Proper validation protects your application from malicious input, enhancing security.

---

By implementing data validation and effective error handling, you'll create a more reliable, secure, and user-friendly application. Ready to put this into practice? Let’s get started! 🎯

## Data Validation and Error Handling

Great progress so far! Now, let's run the code to see data validation and error handling in action.

We have set up validation to ensure that a task cannot be shorter than 5 characters. This is defined in models.py with a custom validator validate_todo_length.

Additionally, we handle errors in our views to provide meaningful messages when user inputs don't meet the validation rules.

Notice that since we have the middleware from the previous unit, the add-todo endpoint now requires a valid Authorization header with a token value to access it.

Running this code will help you understand how these pieces fit together and ensure data integrity and a smooth user experience.

```python
from django.db import models
from django.contrib.auth.models import User

def validate_todo_length(value):
    if len(value) < 5:
        raise ValidationError('Task must be at least 5 characters long.')

class Todo(models.Model):
    task = models.CharField(max_length=200, validators=[validate_todo_length])
    completed = models.BooleanField(default=False)
    user = models.ForeignKey(User, on_delete=models.CASCADE)

    def __str__(self):
        return self.task


from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token
import json
from .models import Todo

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the CSRF token in the response. For simplicity, we are using a fixed token value.
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        task = data.get('task')

        if not task:
            return JsonResponse({'error': 'Task is required'}, status=400)

        new_todo = Todo.objects.create(task=task, user=request.user)

        try:
            new_todo.full_clean()
            new_todo.save()
            return JsonResponse({'message': 'Todo added successfully'}, status=201)
        except ValidationError as e:
            return JsonResponse({'error': e.message_dict}, status=400)
    return JsonResponse({'message': 'Invalid request'}, status=400)


from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)


import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')

# Add a Todo item with user and csrf token
headers = {'Authorization': f'Token {csrf_token}'}

task = "Buy groceries"

response = session.post(BASE_URL + "add-todo/", json={"task": task}, headers=headers)
print(f"Add Todo Response: {response.status_code} - {response.json()}")


from django.urls import path
from myapp import views

urlpatterns = [
    path('register/', views.register, name='register'),
    path('login/', views.user_login, name='login'),
    path('logout/', views.user_logout, name='logout'),
    path('add-todo/', views.add_todo, name='add_todo'),
]

```

### Running the Code: Data Validation and Error Handling

In this exercise, we will validate the behavior of the **To-Do** application by running the code to see data validation and error handling in action. Below is an explanation of the key components and steps required:

---

### Key Components:

1. **Data Validation:**
   - The `validate_todo_length` function ensures that tasks cannot be shorter than 5 characters.
   - The `Todo` model includes this validation, and attempts to create shorter tasks will raise an error.

2. **Error Handling:**
   - In the `add_todo` view, we handle validation errors gracefully by catching them and returning a meaningful error message to the user.

3. **Middleware:**
   - The **AuthMiddleware** ensures that only authenticated users with a valid token can access protected routes, such as adding a new task.

---

### The Workflow:

#### 1. **Register a New User**

First, the application registers a new user by calling the `/register/` endpoint:

```python
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")
```

- This sends the user details (username, email, and password) to the server.
- If successful, a `201` status code is returned, and the user is added to the database.

#### 2. **Login the User and Retrieve CSRF Token**

Next, the user logs in to get a CSRF token that is required for accessing the protected `add-todo` endpoint:

```python
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')
```

- The server returns a `CSRF token` in the response after a successful login.
- The token is included in the headers for subsequent requests.

#### 3. **Add a To-Do Task**

With the CSRF token, the user can now add a new task by sending a POST request to `/add-todo/`:

```python
headers = {'Authorization': f'Token {csrf_token}'}
task = "Buy groceries"

response = session.post(BASE_URL + "add-todo/", json={"task": task}, headers=headers)
print(f"Add Todo Response: {response.status_code} - {response.json()}")
```

- The request sends the task (`Buy groceries`) along with the CSRF token in the headers.
- The `add_todo` view checks if the task length meets the validation rules.
- If the task is valid, a `201` status code is returned with a success message.
- If invalid, a `400` status code and error message are returned.

---

### Expected Output:

1. **Register Response:**
   - If registration succeeds:
     ```bash
     Register Response: 201 - {'message': 'User registered successfully'}
     ```

2. **Login Response:**
   - If login succeeds and CSRF token is provided:
     ```bash
     Login Response: 200 - {'message': 'User logged in successfully', 'csrf_token': 'abc123'}
     ```

3. **Add Todo Response:**
   - If the task meets the validation rules (e.g., `Buy groceries` is longer than 5 characters):
     ```bash
     Add Todo Response: 201 - {'message': 'Todo added successfully'}
     ```

   - If the task is shorter than 5 characters (e.g., `Hi`), the response will look like:
     ```bash
     Add Todo Response: 400 - {'error': {'task': ['Task must be at least 5 characters long.']}}
     ```

---

### Conclusion:

This example demonstrates the integration of **data validation** and **error handling** with **user authentication**. By following this workflow, you ensure that only authenticated users can access protected routes, tasks meet the specified validation criteria, and meaningful feedback is provided to the user when errors occur.

## Enhance Data Validation

Awesome progress! Let's enhance our data validation by adding a check to ensure that a task does not contain numeric values.

Change the myapp/models.py file to add a new validator function that checks if a task contains numeric values. The function should raise a ValidationError if the task contains any numeric values.


```python
from django.db import models
from django.contrib.auth.models import User
from django.core.exceptions import ValidationError

def validate_todo_length(value):
    if len(value) < 5:
        raise ValidationError('Task must be at least 5 characters long.')

# TODO: Define a new validator function to check if a task contains numeric values

class Todo(models.Model):
    # TODO: Add the new validator function to the task field
    task = models.CharField(max_length=200, validators=[validate_todo_length])
    completed = models.BooleanField(default=False)
    user = models.ForeignKey(User, on_delete=models.CASCADE)

    def __str__(self):
        return self.task


from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token
from django.core.exceptions import ValidationError
import json
from .models import Todo

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the CSRF token in the response. For simplicity, we are using a fixed token value.
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        task = data.get('task')

        if not task:
            return JsonResponse({'error': 'Task is required'}, status=400)

        new_todo = Todo.objects.create(task=task, user=request.user)

        try:
            new_todo.full_clean()
            new_todo.save()
            return JsonResponse({'message': 'Todo added successfully'}, status=201)
        except ValidationError as e:
            return JsonResponse({'error': e.message_dict}, status=400)
    return JsonResponse({'message': 'Invalid request'}, status=400)


from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)


import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')

# Add a Todo item with user and csrf token
headers = {'Authorization': f'Token {csrf_token}'}

task = "Buy groceries"

response = session.post(BASE_URL + "add-todo/", json={"task": task}, headers=headers)
print(f"Add Todo Response: {response.status_code} - {response.json()}")

invalid_task = "say hello 1"
response = session.post(BASE_URL + "add-todo/", json={"task": invalid_task}, headers=headers)
print(f"Add Todo Response: {response.status_code} - {response.json()}")


from django.urls import path
from myapp import views

urlpatterns = [
    path('register/', views.register, name='register'),
    path('login/', views.user_login, name='login'),
    path('logout/', views.user_logout, name='logout'),
    path('add-todo/', views.add_todo, name='add_todo'),
]

```

To enhance the data validation and ensure that tasks do not contain numeric values, we can add a custom validator function that checks for the presence of numeric characters in the task. Here’s how you can modify the code:

### Step 1: Define the New Validator in `models.py`

In the `myapp/models.py` file, add a new validator function `validate_no_numbers` to ensure that a task does not contain any numeric characters. Update the `Todo` model to include both the existing and the new validation rules.

```python
from django.db import models
from django.contrib.auth.models import User
from django.core.exceptions import ValidationError
import re

def validate_todo_length(value):
    if len(value) < 5:
        raise ValidationError('Task must be at least 5 characters long.')

def validate_no_numbers(value):
    if any(char.isdigit() for char in value):
        raise ValidationError('Task cannot contain numeric values.')

class Todo(models.Model):
    task = models.CharField(
        max_length=200, 
        validators=[validate_todo_length, validate_no_numbers]
    )
    completed = models.BooleanField(default=False)
    user = models.ForeignKey(User, on_delete=models.CASCADE)

    def __str__(self):
        return self.task
```

### Explanation:
- The `validate_no_numbers` function checks each character in the `task` value to see if it is numeric (`isdigit()`). If any numeric value is found, a `ValidationError` is raised with the message "Task cannot contain numeric values."
- The `Todo` model now has both `validate_todo_length` and `validate_no_numbers` in its `task` field validators, ensuring the task is at least 5 characters long and does not contain numbers.

---

### Step 2: Handle Validation in the View (`add_todo`)

The existing error handling in the `add_todo` view already covers the validation process, so no changes are necessary here. When the new `validate_no_numbers` function raises a `ValidationError`, the error will be caught and handled gracefully.

---

### Step 3: Testing the Validation

Now that the validator has been added, let's test the functionality. Below is the section of the script that checks the new validation rule:

```python
# Add a valid Todo item
task = "Buy groceries"
response = session.post(BASE_URL + "add-todo/", json={"task": task}, headers=headers)
print(f"Add Todo Response (Valid): {response.status_code} - {response.json()}")

# Attempt to add an invalid task with a numeric value
invalid_task = "say hello 1"
response = session.post(BASE_URL + "add-todo/", json={"task": invalid_task}, headers=headers)
print(f"Add Todo Response (Invalid): {response.status_code} - {response.json()}")
```

### Expected Output:

1. **For a valid task (`Buy groceries`)**:
   - The response should indicate success with a `201` status code and a success message:
     ```bash
     Add Todo Response (Valid): 201 - {'message': 'Todo added successfully'}
     ```

2. **For an invalid task (`say hello 1`)**:
   - Since the task contains a numeric value (`1`), the validation should fail, and a `400` status code will be returned with an appropriate error message:
     ```bash
     Add Todo Response (Invalid): 400 - {'error': {'task': ['Task cannot contain numeric values.']}}
     ```

---

### Conclusion:

This enhancement adds another layer of validation by ensuring that the task description does not include any numeric values. This makes the task input more robust, ensuring that it adheres to predefined rules.

## Improve Data Validation in To-Do

Great progress so far!

Now let's add some validation to the Todo model. We want to ensure that the task is at least 5 characters long and at most 50 characters long.

Follow the instructions provided in the myapp/models.py file to complete the task.

```python
from django.db import models
from django.core.exceptions import ValidationError
from django.contrib.auth.models import User

# TODO: Add a custom validator to ensure that the task is at least 5 characters long and at most 50 characters long.
    
    # TODO: If either of the conditions is not met, raise a ValidationError with the message 'Task must be between 5 and 50 characters long.'

class Todo(models.Model):
    # TODO: Add the custom validator to the task field.
    task = models.CharField(max_length=200)
    completed = models.BooleanField(default=False)
    user = models.ForeignKey(User, on_delete=models.CASCADE)

    def __str__(self):
        return self.task



from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token
from django.core.exceptions import ValidationError
import json
from .models import Todo

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the CSRF token in the response. For simplicity, we are using a fixed token value.
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        task = data.get('task')

        if not task:
            return JsonResponse({'error': 'Task is required'}, status=400)

        new_todo = Todo.objects.create(task=task, user=request.user)

        try:
            new_todo.full_clean()
            new_todo.save()
            return JsonResponse({'message': 'Todo added successfully'}, status=201)
        except ValidationError as e:
            return JsonResponse({'error': e.message_dict}, status=400)
    return JsonResponse({'message': 'Invalid request'}, status=400)


from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)


import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')

# Add a Todo item with user and csrf token
headers = {'Authorization': f'Token {csrf_token}'}

task = "Buy groceries"
response = session.post(BASE_URL + "add-todo/", json={"task": task}, headers=headers)
print(f"Add Todo Response: {response.status_code} - {response.json()}")

task = "Buy"
response = session.post(BASE_URL + "add-todo/", json={"task": task}, headers=headers)
print(f"Add Todo Response: {response.status_code} - {response.json()}")


from django.urls import path
from myapp import views

urlpatterns = [
    path('register/', views.register, name='register'),
    path('login/', views.user_login, name='login'),
    path('logout/', views.user_logout, name='logout'),
    path('add-todo/', views.add_todo, name='add_todo'),
]


```

To add the validation for the task length in the `Todo` model, follow these steps:

1. **Define a Custom Validator**:
   We'll create a custom validator to ensure the task's length is between 5 and 50 characters. If the condition is not met, a `ValidationError` will be raised.

2. **Apply the Validator**:
   We'll attach this custom validator to the `task` field in the `Todo` model.

Here’s the implementation:

### 1. Add Custom Validator
We’ll define a function that checks the length of the task string.

```python
from django.core.exceptions import ValidationError

# Custom validator for task length
def validate_task_length(value):
    if len(value) < 5 or len(value) > 50:
        raise ValidationError('Task must be between 5 and 50 characters long.')
```

### 2. Update the `Todo` Model
Now, we’ll attach the custom validator to the `task` field in the `Todo` model.

```python
from django.db import models
from django.core.exceptions import ValidationError
from django.contrib.auth.models import User

class Todo(models.Model):
    # Apply the custom validator to the task field
    task = models.CharField(max_length=200, validators=[validate_task_length])
    completed = models.BooleanField(default=False)
    user = models.ForeignKey(User, on_delete=models.CASCADE)

    def __str__(self):
        return self.task
```

### 3. Handle Validation in the `add_todo` View
In the `add_todo` view, the `full_clean()` method already validates the model fields. If the validation fails, it returns a `ValidationError`.

```python
@csrf_exempt
def add_todo(request):
    if request.method == 'POST':
        data = json.loads(request.body)
        task = data.get('task')

        if not task:
            return JsonResponse({'error': 'Task is required'}, status=400)

        new_todo = Todo(task=task, user=request.user)

        try:
            new_todo.full_clean()  # This will trigger the custom validation
            new_todo.save()
            return JsonResponse({'message': 'Todo added successfully'}, status=201)
        except ValidationError as e:
            return JsonResponse({'error': e.message_dict}, status=400)
    return JsonResponse({'message': 'Invalid request'}, status=400)
```

### Conclusion
By implementing the custom validator `validate_task_length` and attaching it to the `task` field in the `Todo` model, we ensure that the task meets the length requirements (5–50 characters). The `add_todo` view is already equipped to handle validation errors, making this a complete solution for the task length validation.